In [ ]:
import pyarrow.parquet as pq
import numpy as np 
import pandas as pd

In [ ]:
parquet_file = pq.ParquetFile('E2E_Regression.parquet.9')

In [ ]:
X_train = []
for i in range(5139, 6423):
  X_train.append(parquet_file.read_row_group(i).column(9))

In [ ]:
X_train = np.asarray(X_train)
X_train = np.squeeze(X_train, axis=1)
print(X_train.shape)

(1284,)


In [ ]:
print(X_train[2][2][124][124])

0.0


In [ ]:
x1 = []
for i in range(1284):
  for j in range(4):
    for k in range(125):
      x1.append(X_train[i][j][k][:])

In [ ]:
x1 = np.asarray(x1)
print(x1.shape)
print(x1.dtype)
print(x1[0].dtype)
# print(X_train[0][0][0][0].dtype)

(642000, 125)
float64
float64


In [ ]:
np.ndarray.tofile(x1, 'X_val3.dat')

In [ ]:
parquet_file.read_row_group(0).column(3).to_pandas()

0    3.958357
Name: am, dtype: float64

In [ ]:
Y_train = []
for j in range(5139, 6423):
  Y_train.append(parquet_file.read_row_group(j).column(3))

In [ ]:
Y_train=np.asarray(Y_train)
Y_train = np.squeeze(Y_train, axis=1)
print(Y_train.shape)
print(Y_train[0])

(1284,)
3.789943218231201


In [ ]:
# y=np.asarray(y)
# y = np.squeeze(y, axis=1)
print(Y_train.shape)
print(Y_train[0])
print(Y_train.dtype)

(1284,)
3.789943218231201
float64


In [ ]:
np.ndarray.tofile(Y_train, 'Y_train3.dat')

In [ ]:
np.ndarray.tofile(Y_train, 'Y_val3.dat')

In [ ]:
X_train=np.fromfile('X_train3.dat', dtype=np.float64)
X_train=np.reshape(X_train, (5139, 125, 125, 4))
print(X_train.shape)
Y_train=np.fromfile('Y_train3.dat', dtype=np.float64)
print(Y_train.shape)
X_val=np.fromfile('X_val3.dat', dtype=np.float64)
X_val=np.reshape(X_val, (1284, 125, 125, 4))
print(X_val.shape)
Y_val=np.fromfile('Y_val3.dat', dtype=np.float64)
print(Y_val.shape)

(5139, 125, 125, 4)
(5139,)
(1284, 125, 125, 4)
(1284,)


In [ ]:
from __future__ import print_function, division

# from keras.datasets import mnist
from keras.applications.vgg19 import VGG19
from keras.layers import BatchNormalization, Activation
from keras.layers import Input
# from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop,Adam,SGD
from functools import partial

import keras.backend as K
from scipy.ndimage import zoom  

import matplotlib.pyplot as plt

import sys

import numpy as np
import glob
import cv2
from keras.layers import Lambda

In [ ]:
from __future__ import print_function
from __future__ import absolute_import

import warnings
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras import layers
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Conv2D,UpSampling2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D,Add
from keras.layers import GlobalMaxPooling2D
from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
# from keras_applications.imagenet_utils import _obtain_input_shape
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [ ]:
ip=Input(shape=(125, 125, 4))
 
x = layers.Conv2D(1024, (3, 3), activation='relu', padding='same', name='block5d_conv4')(ip)
x=BatchNormalization(momentum=0.8)(x)
        
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4d_conv4')(x)
x=BatchNormalization(momentum=0.8)(x)
        
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block3d_conv4')(x)
x=BatchNormalization(momentum=0.8)(x)
        
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block2d_conv2')(x)
x=BatchNormalization(momentum=0.8)(x)
        
x = layers.Conv2D(254, (3, 3), activation='relu', padding='same', name='block1d_conv2')(x)
x=BatchNormalization(momentum=0.8)(x)

x=layers.Conv2D(3, (3, 3), activation='relu', padding='same')(x)
x=layers.Flatten()(x)
out=layers.Dense(1, activation="linear")(x)
model=Model(ip,out)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 125, 125, 4)]     0         
_________________________________________________________________
block5d_conv4 (Conv2D)       (None, 125, 125, 1024)    37888     
_________________________________________________________________
batch_normalization (BatchNo (None, 125, 125, 1024)    4096      
_________________________________________________________________
block4d_conv4 (Conv2D)       (None, 125, 125, 512)     4719104   
_________________________________________________________________
batch_normalization_1 (Batch (None, 125, 125, 512)     2048      
_________________________________________________________________
block3d_conv4 (Conv2D)       (None, 125, 125, 512)     2359808   
_________________________________________________________________
batch_normalization_2 (Batch (None, 125, 125, 512)     2048  

In [ ]:
opt = Adam(learning_rate=0.0001)
# opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(optimizer=opt,loss='mse',metrics=['mape', 'mse', 'mae'])

In [ ]:
model.load_weights('t3_30.h5')
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=32,epochs=30,verbose=1)

Epoch 1/30
161/161 [==============================] - 1127s 7s/step - loss: 0.0678 - mape: 4.9094 - mse: 0.0678 - mae: 0.1878 - val_loss: 0.0163 - val_mape: 2.7256 - val_mse: 0.0163 - val_mae: 0.1054
Epoch 2/30
161/161 [==============================] - 1033s 6s/step - loss: 0.0112 - mape: 2.2370 - mse: 0.0112 - mae: 0.0856 - val_loss: 0.0125 - val_mape: 2.4472 - val_mse: 0.0125 - val_mae: 0.0937
Epoch 3/30
161/161 [==============================] - 1033s 6s/step - loss: 0.0101 - mape: 2.1115 - mse: 0.0101 - mae: 0.0808 - val_loss: 0.2167 - val_mape: 11.8671 - val_mse: 0.2167 - val_mae: 0.4515
Epoch 4/30
161/161 [==============================] - 1034s 6s/step - loss: 0.0141 - mape: 2.4865 - mse: 0.0141 - mae: 0.0951 - val_loss: 0.0714 - val_mape: 6.2472 - val_mse: 0.0714 - val_mae: 0.2416
Epoch 5/30
161/161 [==============================] - 1034s 6s/step - loss: 0.0095 - mape: 2.0589 - mse: 0.0095 - mae: 0.0787 - val_loss: 0.0318 - val_mape: 3.9013 - val_mse: 0.0318 - val_mae: 0.1472

In [ ]:
model.load_weights('t3_60.h5')
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=32,epochs=30,verbose=1)

Epoch 1/30
161/161 [==============================] - 1121s 7s/step - loss: 0.0090 - mape: 1.9860 - mse: 0.0090 - mae: 0.0760 - val_loss: 0.0249 - val_mape: 3.3310 - val_mse: 0.0249 - val_mae: 0.1292
Epoch 2/30
161/161 [==============================] - 1026s 6s/step - loss: 0.0052 - mape: 1.5176 - mse: 0.0052 - mae: 0.0581 - val_loss: 0.0501 - val_mape: 5.0102 - val_mse: 0.0501 - val_mae: 0.1941
Epoch 3/30
161/161 [==============================] - 1027s 6s/step - loss: 0.0064 - mape: 1.6606 - mse: 0.0064 - mae: 0.0635 - val_loss: 0.0148 - val_mape: 2.6262 - val_mse: 0.0148 - val_mae: 0.1004
Epoch 4/30
161/161 [==============================] - 1027s 6s/step - loss: 0.0046 - mape: 1.4081 - mse: 0.0046 - mae: 0.0539 - val_loss: 0.0492 - val_mape: 4.9384 - val_mse: 0.0492 - val_mae: 0.1913
Epoch 5/30
161/161 [==============================] - 1029s 6s/step - loss: 0.0057 - mape: 1.5680 - mse: 0.0057 - mae: 0.0600 - val_loss: 0.0978 - val_mape: 7.4970 - val_mse: 0.0978 - val_mae: 0.2894


In [ ]:
import keras
print(keras.__version__)

2.4.3


In [ ]:
model.save_weights('t3_90.h5')